In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/"

In [0]:
import numpy as np 
import pandas as pd 
import os

In [4]:
data2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/heart3.csv')
pd.set_option('display.max_colwidth',-1)
print(type(data2))
subsetDataFrame = data2[data2['12'] != '?']

print(len(subsetDataFrame))
subsetDataFrame = subsetDataFrame[subsetDataFrame['13'] != '?']
data=subsetDataFrame
print(len(subsetDataFrame))
print(len(data))
print(data2.head())
#w['female'] = w['female'].map({'female': 1, 'male': 0})
data2['14']=data2['14'].map({0:0,1:1,2:1,3:1,4:1})
data['14']=data2['14'].map({0:0,1:1,2:1,3:1,4:1})


<class 'pandas.core.frame.DataFrame'>
298
296
296
      1    2    3      4      5    6    7      8    9   10   11   12   13  14
0  67.0  1.0  4.0  160.0  286.0  0.0  2.0  108.0  1.0  1.5  2.0  3.0  3.0  2 
1  67.0  1.0  4.0  120.0  229.0  0.0  2.0  129.0  1.0  2.6  2.0  2.0  7.0  1 
2  37.0  1.0  3.0  130.0  250.0  0.0  0.0  187.0  0.0  3.5  3.0  0.0  3.0  0 
3  41.0  0.0  2.0  130.0  204.0  0.0  2.0  172.0  0.0  1.4  1.0  0.0  3.0  0 
4  56.0  1.0  2.0  120.0  236.0  0.0  0.0  178.0  0.0  0.8  1.0  0.0  3.0  0 


In [5]:
X=data.loc[:,:'13'].values
#print(X.shape)
#print(type(X))
np.delete(X, 86, axis=0)
print(X.shape)
#print(X[87][12])
#rint(float(X[1][12]))
#X=[]
for i in range(295):
  y1=X[i][11]
  y2=X[i][12]
  if(y1=='?'):
    np.delete(X, (i), axis=0)
    print(i)
  elif(y2=='?'):
    np.delete(X, (i), axis=0)
    print(i)
  else:
    X[i][11]=float(X[i][11])
    X[i][12]=float(X[i][12])
  
  

#X.head()

(296, 13)


In [6]:
print(X.shape)
print(X[1][11])
#print(y_label.shape)

(296, 13)
2.0


In [7]:
import tensorflow as tf

y=data.loc[:,'14'].values
print(len(y))
print(y[1])
print(y[2])
pos=0
neg=0
mixed=0
neutral=0
count=0
z=[]
for i in y:
  if i== 0:
    pos=pos+1
    
  else:
    neg=neg+1
  


print(pos)
print(neg)


print(y[1])    
y_label=tf.keras.utils.to_categorical(y,num_classes=2)
#prin

296
1
0
159
137
1


In [10]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y):
    Cs = [0.01, 0.1, 1 ,10]
    gammas = [1, 0.1, 0.01, 10]
    degrees=[4, 3, 5]
    kernels=['rbf','poly','linear']
    param_grid = {'C': Cs,'degree':degrees,'gamma':gammas}
    
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_
  
grid=svc_param_selection(X_train,X_label)
print(grid)  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'C': 10, 'degree': 4, 'gamma': 1}


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, X_label, X_test_label = train_test_split(X,y, test_size=0.1)
print(len(X_train))
print(len(X_label.shape))
#print(X_label.columns)
print(len(X_test))
print(len(X_test_label))

266
1
30
30


In [12]:
from sklearn.svm import SVC
cgx = SVC(C=10,kernel='linear',degree=4,gamma='auto',probability=True)
print(len(X_train))
print(len(X_label))
cgx.fit(X_train, X_label)



266
266


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=4, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
from sklearn.metrics import accuracy_score
accuracy=cgx.score(X_train,X_label)
print(accuracy)
accuracy2=cgx.score(X_test,X_test_label)
y_pred4 = cgx.predict(X_test)
accuracy = accuracy_score(y_pred4, X_test_label)
print(accuracy)
#print(accuracy2)

0.8533834586466166
0.8333333333333334


In [16]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.08
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=X_label)
d_valid = xgb.DMatrix(X_test, label=X_test_label)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=2000, verbose_eval=10)

[0]	train-logloss:0.649483	valid-logloss:0.651175
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 2000 rounds.
[10]	train-logloss:0.407985	valid-logloss:0.461799
[20]	train-logloss:0.293417	valid-logloss:0.412901
[30]	train-logloss:0.228659	valid-logloss:0.396611
[40]	train-logloss:0.189019	valid-logloss:0.412683
[50]	train-logloss:0.161768	valid-logloss:0.422897
[60]	train-logloss:0.140935	valid-logloss:0.431314
[70]	train-logloss:0.125051	valid-logloss:0.446239
[80]	train-logloss:0.111147	valid-logloss:0.458184
[90]	train-logloss:0.099853	valid-logloss:0.469912
[100]	train-logloss:0.089947	valid-logloss:0.480037
[110]	train-logloss:0.082594	valid-logloss:0.481893
[120]	train-logloss:0.076139	valid-logloss:0.488015
[130]	train-logloss:0.070418	valid-logloss:0.497935
[140]	train-logloss:0.06609	valid-logloss:0.510191
[150]	train-logloss:0.061997	valid-logloss:0.516576
[160]	train-logloss:0.05872

In [15]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# load data

# fit model no training data
model = XGBClassifier()
model.fit(X_train, X_label)
# make predictions for test data
y_pred1 = model.predict(X_test)

# evaluate predictions
accuracy = accuracy_score(X_test_label, y_pred1)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.00%
